---
title: American Community Survey
jupyter:
  jupytext:
    cell_metadata_filter: '-all'
    main_language: python
    notebook_metadata_filter: '-all'
  kernelspec:
    display_name: Python 3 (ipykernel)
    language: python
    name: python3
---



The American Community Survey (ACS) is a large survey of households
and individuals in the United States, carried out by the US
government on a continuous basis (around 3.5 million people are
contacted per year).  It is arguably the most authoritative source
of information about the demographic composition of the US
population, and is used for many purposes in academic research,
government, public policy, and in private industry.

Some of the questions in the ACS are about sensitive topics, and therefore are only released in aggregate form. The "public use microsample" (PUMS) is a set of individual ACS responses that only includes information that has been deemed safe for public release at the individual level. Here we will work with a subset of the ACS/PUMS data.

You will need to refer to the documentation to know what the ACS variable names mean: **[ACS PUMS Codebooks](https://www.census.gov/programs-surveys/acs/microdata/documentation/2018.html)** Scroll down for data dictionary 2018 "1-year" ACS/PUMS, available in several formats.

For this course, we are providing a simplified version of the ACS/PUMS data from 2018. It contains a random subset of the cases and a selected subset of variables.

Note that many PUMS variables are described as being "household" or "individual" variables. These refer to characteristics of households (one or more people living at the same address) or to characteristics of individual people, respectively.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sb
import scipy.stats as sps
import os

Next we load the data from the filesystem into a dataframe.


In [ ]:
df = pd.read_csv("./pums_short.csv.gz")

In [ ]:
df.columns

## Missing values

Most datasets in the real world have missing values.  There are many
reasons that a value may be missing.  For example, in some cases it
makes no sense to calculate a number (e.g. income for a young
child), or a person may refuse to answer a question in a survey, or
a value may have inadvertently not been collected.

In Pandas, missing values are represented using the symbol `NaN` (we can refer to it in programs using `np.nan` if we need to),
which means "not a number".  The method `isnull` tells us which
values in a Pandas data structure (a dataframe or series) are missing ("null" is a computer science term of missing or "not yet recorded").

In [ ]:
s_with_nan = pd.Series([1, np.nan, 2, 10])
s_with_nan

In [ ]:
s_with_nan.isnull()

Recall that if we take the mean of boolean values we get a proportion:

In [ ]:
s_with_nan.isnull().mean()

If we want to know the proportion of observations for each variable
that are missing, we can use `isnull` and `mean` on an entire table. Do so for the `df` table.


<details>

```
df.isnull().mean()
```
    
</details>

The above results shows us that a few variables, e.g. RNTP, have
many missing values, while others, e.g. DIVISION, have no missing
values.  The RNTP variable contains information about the amount of
rent someone pays, so is always missing if a person does not rent
the place where they live.  DIVISION indicates where in the US the
person lives.  DIVISION can never be missing because the census
bureau always has this information about each respondent.

If you want to drop all cases (rows) of a variable that are missing,
you can use the `dropna` method.  For example, suppose we want the
non-missing values for the amount of money that households pay for
rent (RNTP).  As noted above, these values are missing for
households that do not rent their place of residence.  We can use
the following code to obtain the non-missing values and store them
in a series called x:

In [ ]:
x = df["RNTP"].dropna()

If is often useful to "chain" `dropna` with other methods or
attributes of a series.  For example, if we want to know how many
people have a non-missing rent value (i.e. are renters), we can use

In [ ]:
x.size

The treatment of missing values in a statistical analysis is a
complex topic.  Here we are simply demonstrating how to drop the
missing values from the dataset.  Sometimes this is the correct
thing to do, but often it is not.

## Summary statistics of income

We will turn now to the variable "HINCP" which is the household
income -- the combined income of everyone living in one household.
Note that income is not the same thing as wealth.  Many people have
far more wealth than income.

Overall, the United States has the following income characteristics:

In [ ]:
df["HINCP"].describe()

Let us observe two things:

1. The summary statistics above are computed using the non-missing
values of HINCP.  Most Pandas dataframe methods, like `describe`,
automatically drop missing values, but functions and methods from
other Python packages generally do not, so in some settings you will
need to drop the missing values explicitly using `dropna`.

2. The data are printed out in "scientific notation" such that `3e+02` means $3 \cdot 10^2 = 300$. To make this a little easier to read, let's rewrite this as "income in thousands of dollars":

In [ ]:
df["HINCP_1000"] = df["HINCP"] / 1000
df["HINCP_1000"].describe()

Use a `histplot` or `kdeplot` to show the distribution of `"HINCP_1000"`. Observe the the connection with location (`mean` and `50%`, the median) and variation (`std` and the difference of `25%` and `75%`).


You will notice that a very small fraction of the household income
values are negative, as shown below.  This is not an error.  Using
standard definitions for income, it is possible for a household (or
an individual) to have negative income.  Compute the proportion of
households with negative income. (Hint: remember that a proportion is the mean of a boolean series).


<details>
    
```    
(df["HINCP_1000"] < 0).mean()
```
                        
</details>

## Spread and Skew

We already went through a couple measures of spread, even if we did not realize! Let's check again:

In [ ]:
df["HINCP_1000"].describe()

Apart from the mean, what gives us an idea of how spread out or skewed our data actually is?  In other words, how can other quantities help in giving us an idea of the behavior of the data like spread and skew?

### Spread

We will be taking a look at different measures of spread in order to quantify the variation within our data. In particular, we will be looking at three measures of spread:

- **Range**: This is the simplest measure of spread, calculated as the difference between the maximum and minimum values.
- **Standard deviation**: Measures the variation of values of a variable about their mean.
- **Quantiles**: Divide data into regions of equal probability, such as quartiles or percentiles, which can give you insights on the distribution of the data. From these measures of spread, quantiles are the most robust to outliers.

#### Range

In [ ]:
income_min = df["HINCP"].min()
income_max = df["HINCP"].max()
income_range = income_max - income_min
print(f"Minimum Income: {income_min}, Maximum Income: {income_max}, Range: {income_range}")

The range by itself does not tell the full story, as this is a measure of spread that is specially sensitive to outliers. For instance, a single very high income could make the range extremely large, while the rest of the values are closer on average.

#### Standard Deviation

The standard deviation (and the variance!) measure spread by calculating the average distance of the data points from the mean. 

In [ ]:
income_std = df["HINCP"].std()
print(f"Standard Deviation of Household Income: {income_std}")

The standard deviation, conveniently, retains the same units as the data, making it intuitive, unlike the **variance**, which is the square of the standard deviation. Although it is useful in theoretical contexts, the variance is harder to interpret in practice.

In [ ]:
income_var = df["HINCP"].var()
print(f"Variance of Household Income: {income_var}")

High values of the standard deviation means that the data points are widely spread out, whereas low values indicate that the data is clustered together.

Let's see how income data spreads one standard deviation around the mean.

In [ ]:
income_mean = df["HINCP"].mean()
income_std = df["HINCP"].std()

# Let's create the histogram plot with the KDE
ax = sb.histplot(df["HINCP"], kde=True, color='blue')

# This code shades the area within one standard deviation around the mean
xmin, xmax = income_mean - income_std, income_mean + income_std
ax.axvline(income_mean, color='red', linestyle='--', label='Mean')
ax.fill_betweenx([0, ax.lines[0].get_data()[1].max()], xmin, xmax, color='yellow', alpha=0.3, label='Within 1 Std Dev')

# Always add labels and legend!
ax.set_title('Distribution of Household Income (HINCP) with One Std Dev Shaded')
ax.set_xlabel('Household Income')
ax.set_ylabel('Density')
ax.legend()

sb.despine()

#### Quantiles

Quantiles divide observations into intervals of equal probability, and they are more robust against outliers. 

In [ ]:
# Plotting the distribution of income with quantiles
ax = sb.histplot(df["HINCP"], kde=True, color='blue')

# Add vertical lines for Q1, median (Q2), and Q3
Q1 = df["HINCP"].quantile(0.25)
median = df["HINCP"].quantile(0.50)
Q3 = df["HINCP"].quantile(0.75)

ax.axvline(Q1, color='black', linestyle='--', label='Q1 (25th Percentile or 1st Quartile)')
ax.axvline(median, color='red', linestyle='--', label='Median (50th Percentile or 2nd Quartile)')
ax.axvline(Q3, color='black', linestyle='--', label='Q3 (75th Percentile or 3rd Quartile)')

ax.set_title('Household Income Distribution with Quartiles')
ax.set_xlabel('Household Income')
ax.set_ylabel('Density')
ax.legend()

sb.despine()

Let's take a look at the corresponding boxplot fror Household Income.

In [ ]:
# Creating the boxplot
ax = sb.boxplot(x=df["HINCP"], color='lightblue')

# Adding labels and title
ax.set_title('Boxplot of Household Income (HINCP)')
ax.set_xlabel('Household Income')

sb.despine()

**Thought problem:** What is the correspondence between these two plots?


**Lab problem:** Compute the $16$th and the $84$th percentiles and


<details>
    
**Answer**:

This is how we find quantiles using pandas:

In [ ]:
P16 = df["HINCP"].quantile(0.16)
P84 = df["HINCP"].quantile(0.84)

print(f"16th Percentile: {P16}")
print(f"84th Percentile: {P84}")

We can overlay our previous histogram showing one standard deviation about the mean and the 84th and 16th percentile. 

In [ ]:
ax = sb.histplot(df["HINCP"], kde=True, color='blue')

ax.axvline(P16, color='green', linestyle='--', label='16th Percentile (P16)')
ax.axvline(P84, color='green', linestyle='--', label='84th Percentile (P84)')

xmin, xmax = income_mean - income_std, income_mean + income_std
ax.axvline(income_mean, color='red', linestyle='--', label='Mean')
ax.fill_betweenx([0, ax.lines[0].get_data()[1].max()], xmin, xmax, color='yellow', alpha=0.3, label='Within 1 Std Dev')



ax.set_title('Household Income Distribution with 68% Range Shaded')
ax.set_xlabel('Household Income')
ax.set_ylabel('Density')
ax.legend()

</details>

### Skewnes

Skew or skewness measure the asymmetry of the distribution, and tells us whether the data has longer tails on either side of the distribution. This is important, since many real-world variables are often skewed, and identifying skewness can guide us in our analysis, e.g. selection of methods, applicable transformations, deviation from assumptions, etc...


**Types of Skewness**

- **Symmetrical (No Skew)**: The data is evenly distributed around its mean or median. 
- **Right Skew**: The tail on the right side is longer than that on the left side. In this case, the mean is greater than the median.
*Example*: Income tends to have a right skew. Why?
<details>
Income is right-skewed because most people earn around the average income, but few people earn significantly more.
</details>
- **Left Skew**: The tail on the left is longer thant that on the right and the mean is less than the median.
*Example:* Death of natural causes tends to have a left skewed distribution. Again, why?
<details>
Most people die at an older age, but there are a few cases of death at younger ages, pulling the mean age of death down. 
</details>

Skewness helps us understand the shape of our data and can guide us in our choice of methods. 

As we have come to expect, pandas already has functionality to compute skewness, but before that, do we expect it to be $0$, positive or a negative value? Let's take a look first:

In [ ]:
# Plotting the KDE plot to visualize skewness
ax = sb.kdeplot(df["HINCP"], fill=True, color='blue')
ax.set_title('KDE Plot of Household Income (HINCP)')
ax.set_xlabel('Household Income')
ax.set_ylabel('Density')

# Adding a vertical line for the mean
income_mean = df["HINCP"].mean()
income_median = df["HINCP"].median()
ax.axvline(income_mean, color='red', linestyle='--', label='Mean')
ax.axvline(income_median, color='green', linestyle='--', label='Median')

# Adding a legend
ax.legend()

Now let's compute the value.

In [ ]:
income_skew = df["HINCP"].skew()
print(f"Skewness of Household Income: {income_skew}")

Just as we predicted.

## Stratifying to explain variation

When considering a population such as the United States, which is
extremely **heterogeneous** (contains high amounts of variation), it is usually much more informative to
analyze the data after stratifying the population according to the
values of factors that explain some of the heterogeneity.  The ACS
includes a variable called "FES" which partitions the population
into 8 subgroups based on household structure.  See the data
dictionary for the precise definitions of the groups (note that "LF"
in the documentation stands for "labor force").

Create a box plot that shows the "FES" on the x-axis and "HINCP_1000" on the y-axis (remember that `showfliers = True` can be a useful option).


<details>

```
sb.boxplot(data = df, x = "FES", y = "HINCP_1000", showfliers = False)
```

</details>

We start to see some patterns in the data, but let us make them more precise. We can look at the set of standard summary statistics captured
by the `describe` method, but now restricting the calculations to
the values within each stratum (group defined by FES value).

Use the `groupby` method to group the table on "FES". Use the `describe()` method on the `"HINCP_1000"` column.


<details>

```
df.groupby("FES")["HINCP_1000"].describe()
```

</details>

From the table above, we see that the wealthiest households, based
on the median income, are those including a married couple, with
both members of the couple working (group 1).  The least wealthy
households are those including a single female who is not working
(group 8).

It is often convenient to re-order the rows of the output in order
to sort one of the values.  Repeat the previous use of `describe` but add the `.sort_values(by = "50%")` to sort the data on the median column.


<details>

```
df.groupby("FES")["HINCP_1000"].describe().sort_values(by = "50%")
```

</details>

## Multiple explanatory factors

In many cases we want to stratify on two or more factors that may
"explain" the variation in a value of interest.  Above we considered
household structure as one explanatory factor.  Now we will consider
the geographic region where the respondent lives as well.  The
Census Bureau has several ways of partitioning by geography, the
variable `REGION` here uses four levels (see the data dictionary for
what they correspond to).  We will conduct a "two-way"
stratification, and determine the median income for people living in
each combination of a FES level and a REGION level.

First, we create the grouping and show off the sizes of the group.

In [ ]:
grp = df.groupby(["REGION", "FES"])
grp.size()

Now, we want to compute the median within each group. Run the `median` method on the "HINCP_1000" column of the  `grp` object. Save the result to a variable, but then display it as well.


<details>
    
```
grp_median = grp["HINCP_1000"].median()
grp_median
```
    
</details>

The result of a `groupby` and computation such as `mean` or `median` expression, such as above, is
a new dataframe in which the distinct combinations of the grouping
variables (here, "FES" and "REGION") form the dataframe's _index_.

We can make the index into a set of columns using `reset_index`.

In [ ]:
grp_sizes = grp.size().reset_index()
grp_sizes

This makes it possible to use Seaborn to plot the result.

In [ ]:
sb.barplot(data = grp_sizes, x = "REGION", y = 0, hue = "FES")

Here we see that region 3 is the most populous and that the same basic patterns tend to continue across regions. Perhaps we notice that region 4 (the West) is unusual in that category 2 ("2 .Married-couple family: Husband in labor force, wife not in LF") is more common than category 4 ("4 .Married-couple family: Neither husband nor wife in LF"). This may indicate a generally younger population with fewer retired couples.

We could be more precise if we had a table with each region as a row and each FES category as a column.

To view these results as a table, it is easier to "unstack" the
results, which moves one level of the row index to the columns.
This is also called "pivoting".

In [ ]:
grp_size_tbl = grp.size().unstack()
grp_size_tbl

Now we can ask for which region is category 2 more common than 4?

In [ ]:
grp_size_tbl[2.0] > grp_size_tbl[4.0]

Use the `unstack` method on the grouped medians. Save the result to a variable, but also display it. What are the rows and columns of the result?


<details>
    
```
grp_median = grp["HINCP"].median() # you probably don't have to recreate, but just to be clear

gms = grp_median.unstack()
gms
```
    
</details>

In which categories does region 1 have higher median income than region 4? (Recall the use of `.loc` to select rows of a table.) Use a comparison to find out.


<details>
    
```
gms.loc[1.0] > gms.loc[4.0]
```
    
</details>

This analysis shows that it is possible to gain insight about two
explanatory factors by stratifying the data on the _cross product_
consisting of all subgroups defined by pairs of levels of the two
factors.  These subgroups are sometimes called _cells_.  It should
be clear however that this approach will not "scale" very well --
cross products for three or more factors will generally produce huge
numbers of cells.  We may have too little data per cell to produce
meaningful estimates of population quantities, and even if this is
not an issue, there will be a huge number of combinations to
consider.  The difficulty of _controlling_ for, or stratifying on
multiple explanatory factors is one of the major challenges that
arises in statistical data analysis.
